#Part 1
import requests
import json
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text
import uuid
from datetime import date, datetime, timedelta

def retrieve_text_from_url(url):
  """Remove html tags from a string"""
  try:
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    return soup.get_text()
  except:
    return""


def get_news(keyword):
  newsapi_key = 'ffba3148c6d1447ca0b54113757555e0'
  endpoint = 'https://newsapi.org/v2/everything'
  parameters = {
      'q': keyword,
      'apiKey' : newsapi_key,
      'pagesize' : 100
  }
  data = requests.get(endpoint, params = parameters).json()
  #Get the full text for the articles
  for article in data['articles']:
    article['full_text'] = retrieve_text_from_url(article['url'])
  return data['articles']




Instructions

In this assignment, we will explore how to combine two powerful APIs - NewsAPI and ChatGPT - to create a pipeline that retrieves news articles, analyzes their sentiment and entities, and stores the results in a database. The NewsAPI provides us with access to a wealth of news articles, while ChatGPT allows us to extract valuable insights from these articles. By integrating these APIs together and storing the results in a database, we can create a powerful tool for analyzing and understanding the news. In the following sections, we will walk through the steps required to build this pipeline and explore some of the challenges and opportunities along the way.

Part 1:

Use the NewsAPI to get URLs for news articles. Go to https://newsapi.org, create an account, and get a key. Use the https://newsapi.org/v2/everything endpoint, which inputs a keyword, queries the NewsAPI, and returns a list of URLs with the news stories containing the keyword. Follow the documentation at https://newsapi.org/#documentation and figure out how to get back the news. Similarly, create an account with OpenAI, and sign up to use their API. You will need to enter your credit card info, as the API charges on a per-use basis. (The charges are not high as the usage in this project is going to be rather minimal.)

Part 2:

Write code that stores the retrieved news articles in a database. You can use the db.ipeirotis.org MySQL server and create your table under the public database.

Please prefix with your netID all the tables you create in the public database. So, if you want to create a table called news and your netID is ab123, call the table ab123_news.

You will need to figure out which fields you want to save in the database, and their data types, create the appropriate table, and then insert in the database the news entries that you retrieved in Part 1.

Part 3:

Retrieve the news articles you stored in the database in Part 2, and use the ChatGPT to extract (a) the sentiment of each of the news articles, (b) the entities discussed in the text, and (c) anything else that you want to extract.

Part 4:

Store in the database the sentiment, entities, and the additional element that you extracted from each news article. The sentiment table should contain just two fields, url and sentiment_score. The entities table should contain url and entity. You will probably need a table with a similar structure for whatever you want to extract.

In [4]:
!pip install feedparser
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [14]:
import mysql.connector
def test_connection():
    connection = mysql.connector.connect(
        host='localhost',
        port=8889,   
        user='root',         
        password='root',     
        database='news_articles_db'
    )
    if connection.is_connected():
        print("Connected to the database!")
        connection.close()
    else:
        print("Failed to connect.")

test_connection()


Connected to the database!


In [12]:
def store_article_in_db(article, connection):
    cursor = connection.cursor()
    
    insert_query = """INSERT INTO news_articles (title, summary, link, published)
                      VALUES (%s, %s, %s, %s)"""
    
    title = article['title']
    summary = article['summary']
    link = article['link']
    published = article['published']

    cursor.execute(insert_query, (title, summary, link, published))
    connection.commit()
    cursor.close()

def retrieve_articles_from_db(connection):
    cursor = connection.cursor(dictionary=True)
    
    select_query = "SELECT * FROM articles"
    
    cursor.execute(select_query)
    articles = cursor.fetchall()
    
    cursor.close()
    return articles

In [13]:

import feedparser
import email.utils
from datetime import datetime

#cursor.execute("INSERT INTO new_articles (title, summary, link, published) VALUES (%s, %s, %s, %s)", (article['title'], article['summary'], article['link'], article['published']))

RSS_FEEDS = {
    'technology': 'http://feeds.bbci.co.uk/news/technology/rss.xml',
    'world': 'http://feeds.bbci.co.uk/news/world/rss.xml',
    'business': 'http://feeds.bbci.co.uk/news/business/rss.xml',
    # ... add more topics and URLs as needed
}

def get_rss_news(topic):
    if topic not in RSS_FEEDS:
        print(f"No RSS feed available for topic: {topic}")
        return []

    feed_url = RSS_FEEDS[topic]
    feed = feedparser.parse(feed_url)

    articles = []
    for entry in feed.entries:
        article = {
            "title": entry.title,
            "summary": entry.summary,
            "link": entry.link,
            "published": entry.published
        }
        articles.append(article)
    
    return articles

topic = 'technology' # or 'world', 'business', etc.
articles = get_rss_news(topic)


connection = mysql.connector.connect(
    host='localhost',
    port=8889,  
    user='root',  
    password='root', 
    database='news_articles_db'
)

for article in articles[:50]:
    # Convert the 'published' date from RSS format to YYYY-MM-DD format for SQL
    parsed_date = email.utils.parsedate_to_datetime(article['published'])
    article['published'] = parsed_date.date().isoformat()

    # Store the article in the database
    store_article_in_db(article, connection)

    # Print the article (for debugging)
    print(article['title'])
    print(article['summary'])
    print("Link:", article['link'])
    print("Published:", article['published'])
    print("-------------------------------")

connection.close()


Arm: Chip designer to the world in $54bn market return
Arm shares were priced at the top of the range that had been indicated to prospective investors.
Link: https://www.bbc.co.uk/news/business-66805116?at_medium=RSS&at_campaign=KARANGA
Published: 2023-09-14
-------------------------------
'Overwhelming consensus' on AI regulation - Musk
Tech heavyweights gathered in Washington DC to discuss the regulation of artificial intelligence.
Link: https://www.bbc.co.uk/news/technology-66804996?at_medium=RSS&at_campaign=KARANGA
Published: 2023-09-13
-------------------------------
AI and sound - helping firms build their own 'sonic identity'
Artificial intelligence is assisting companies in developing their own signature sounds.
Link: https://www.bbc.co.uk/news/business-66330890?at_medium=RSS&at_campaign=KARANGA
Published: 2023-09-13
-------------------------------
France halts iPhone 12 sales over radiation levels
Apple has been told it must recall every iPhone 12 sold in the country if it can

In [30]:
!pip install openai
!pip install beautifulsoup4 requests


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [43]:
import openai
import mysql.connector
# Set up the OpenAI API key
openai.api_key = '########'


In [61]:
import mysql.connector
import requests
from bs4 import BeautifulSoup
import openai

MAX_TOKENS = 2048
CONTENT  = ""

def fetch_articles_from_db(query_term=None):
    # Connect to the database
    cursor = connection.cursor(dictionary=True)
    
    if query_term:
        sql = f"SELECT * FROM news_articles WHERE title LIKE %s OR summary LIKE %s"
        cursor.execute(sql, (f"%{query_term}%", f"%{query_term}%"))
    else:
        sql = "SELECT * FROM news_articles"
        cursor.execute(sql)
    
    articles = cursor.fetchall()
    connection.close()
    
    return articles


def fetch_article_content_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = ' '.join([p.get_text() for p in paragraphs])
    return content

#import openai

def ask_chatgpt_for_summary(content):

    prompt = f"Summarize the following article: {content}"
    if len(prompt) > MAX_TOKENS:
        return "Content too long to summarize."

    response = openai.Completion.create(
        model="gpt-3.5-turbo-0301",
        prompt=prompt,
        max_tokens=150  # Adjust based on how long you want the answer to be
    )
    return response.choices[0].text.strip()


def store_summary_in_db(article_id, summary):
    connection = mysql.connector.connect(
        host='localhost',
        port=8889,
        user='root',
        password='root',
        database='news_articles_db'
    )
    cursor = connection.cursor()

    sql = "UPDATE news_articles SET summary_gpt = %s WHERE id = %s"
    cursor.execute(sql, (summary, article_id))

    connection.commit()
    connection.close()


def summarize_articles():
    articles = fetch_articles_from_db("TIKTOK")
    
    for article in articles:
        url = article['link']
        content = fetch_article_content_from_url(url)
        CONTENT = content
        print(content[0:1000000])  # Printing the first 100 characters for checking

        if len(content) > MAX_TOKENS:
            print(f"Content for article '{article['title']}' is too long to summarize.")
            continue

        # Call ChatGPT to summarize
        summary = ask_chatgpt_for_summary(content)
        print(f"Title: {article['title']}\nSummary: {summary}\n")

        # Store the summary in the database
        store_summary_in_db(article['id'], summary)

summarize_articles()
#print(CONTENT)

TikTok has opened its first European data centre to alleviate fears over Chinese state surveillance.  The firm says European users' data is now migrating to servers in Dublin, as part of its ongoing response to data privacy concerns around the video-sharing app's links to China. TikTok, which is owned by Chinese firm ByteDance, says it has never given data to Beijing. Critics fear that the Chinese state could request access anytime.  The video-sharing giant is also allowing a European security company access to audit cyber-security and data protection controls.  TikTok has called this "Project Clover", nodding to the pivotal role that Ireland is playing. It is running in parallel with "Project Texas", which involved promising similar measures to US lawmakers in 2020.  Earlier this year TikTok faced a number of government restrictions on its use on cyber-security and privacy grounds.  A spate of institutions decided to ban the app from officials' devices, including the UK government, th

In [56]:
fetch_articles_from_db("TIKTOK")

[{'id': 28,
  'title': 'TikTok opens Dublin data centre to ease China spying fears',
  'summary': "There is concern over the video-sharing app's links with China and who accesses its users' data.",
  'link': 'https://www.bbc.co.uk/news/technology-66717589?at_medium=RSS&at_campaign=KARANGA',
  'published': datetime.date(2023, 9, 5),
  'summary_gpt': None},
 {'id': 48,
  'title': 'Edinburgh Fringe: Can TikTok comedy stars cut it on stage?',
  'summary': 'They have millions of followers, but can they cut it in front of a crowd at the Edinburgh Fringe?',
  'link': 'https://www.bbc.co.uk/news/entertainment-arts-66569003?at_medium=RSS&at_campaign=KARANGA',
  'published': datetime.date(2023, 8, 23),
  'summary_gpt': None}]

In [49]:
!pip install scikit-learn nltk


Defaulting to user installation because normal site-packages is not writeable


In [79]:
#!pip install gensim
# !pip install gensim --upgrade
!pip install gensim==3.8.3
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

# Make sure you have the punkt tokenizer downloaded
from nltk.tokenize import sent_tokenize
from gensim.summarization import summarize

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 6.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-macosx_10_9_universal2.whl size=24686057 sha256=b68e7264ae5ddfd142c892e9b5011d3f49d21414521c9b66dab436b795aef4b1
  Stored in directory: /Users/mickeyshamah/Library/Caches/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mickeyshamah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mickeyshamah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ImportError: cannot import name 'has_pattern' from 'gensim.utils' (/Users/mickeyshamah/Library/Python/3.9/lib/python/site-packages/gensim/utils.py)

In [71]:
# TF-IDF stands for Term Frequency-Inverse Document Frequency. approch!!!
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string

# If you haven't already, you'll need to download the stopwords and punkt tokenizer models:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in string.punctuation]
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    return ' '.join(tokens)

def extract_key_sentences(text, n_sentences=5):
    sentences = sent_tokenize(text)
    preprocessed_sentences = [preprocess(sent) for sent in sentences]
    
    vectorizer = TfidfVectorizer().fit(preprocessed_sentences)
    tfidf_matrix = vectorizer.transform(preprocessed_sentences)
    sentence_scores = tfidf_matrix.sum(axis=1)
    
    ranked_sentences = [sentences[i] for i in sentence_scores.argsort()[0, -n_sentences:].tolist()[0][::-1]]
    
    return ' '.join(ranked_sentences)

# Test the function

text_sample = "They have millions of followers for their TikTok sketches. But how will four hugely successful creators fare in a live stand-up show at Britains biggest comedy festival, the Edinburgh Fringe? This is one great big Edinburgh experiment, says Coco Sarel as an introduction and possibly a caveat in case things go wrong. Sarel (900,000 followers) is one of four TikTok comedians who, she says, want to find out if they're any good at stand-up. Which is better than four LinkedIn comedians, so youre going to have a decent hour. A good gag. A good start. The poster for their joint Fringe show, titled Knock Knock, boasts that the quartet have 7+ million followers. Its gone up since that was printed. According to their TikTok profiles, they have 8.6 million followers between them, and 365 million likes. There are a couple of hundred people in this bar. Its a tiny audience compared with their online numbers, but being able to hear the laughter (or not) of a crowd is very different from filming something on your phone at home. As host, Sarel gets stuck straight in with some crowd work. In other words, asking people in the front row what they do for a living and trying to come up with an amusing response. But Sarel struggles to find good banter with the first victims and awkwardly moves on. Its not long before we know the professions of most of the people in the first two rows. Sarel does pluck out some good quips, but when she picks on an audience member who works in marketing, the comedian admits defeat. I think crowd work is done! She has natural energy and charisma, and has more success with a relatable routine re-enacting a group chat between the typical members of a female friendship group. As MC, Sarel returns in between the other acts. She grows more comfortable and assured, and proves her crowd work can work when she plays matchmaker in the audience. Its all good-natured and this time the awkwardness is intentional. Steven McKell (3.8 million) is - how to put this? - larger-than-life and flamboyant, with a flair for physical comedy thats attracted attention online. His sashay onto the stage could form a TikTok video on its own, but here it only fills five seconds out of his 15-minute slot. After a high-kick and a questionable claim to be a one-man entertainment machine - like Beyonce if she was from Fife, the rest of his set focuses on his family life. As he tells it, they were poor, they fought, they put one of his eight siblings in a tumble dryer and dangled another out of the bedroom window - but they looked out for each other. When the police broke down the front door in a raid one day, his diminutive but fearsome Scottish mother headbutted one of the officers, he says, right in the shins. He has learned the stand-up trick of starting with a grain of truth and embellishing for comic effect. Hes a big personality, and manages to hold the crowd by doing more than just goofing around. Ayame Ponder (2.7 million) seems to have built her huge following largely by commentating on videos of things like bottles being rolled down stone steps and watching them smash. Yes, thats a thing on TikTok. She starts her set with tongue-in-cheek brags about being a TikTok star and incredibly famous, before moving on to everyday topics like dating and the nicknames shes given boyfriends based on their, er, physical attributes. Shes as likeable and engaging on stage as she is online, and gets the audience on side without setting them alight. Still, she tells them as she exits: Ive been amazing, youve been so-so. Finally, Henry Rowley (1.2 million) made his name with videos parodying pompous posh people. On stage, his history teacher and his dad both sound uncannily like Richard E Grant, and when he says he wants to branch out it turns out that means parodying posh people at music festivals. He puts his whole body into fully acting out his ridiculous characters, and earns extra laughs by being more blunt and risque than his co-stars. Of the four, he has the most fully-formed act, is the most convincing storyteller, and seems to have the self-possession required for stand-up. So did the experiment work? There were mixed results but overall, yes, these TikTok comedians can cut it on stage. They arent the only online creators in Edinburgh - Serena Terry aka Mammy Banter (2.2 million) and Abi Clarke (906,000) are among the biggest TikTok names to have brought their own shows. Stand-up is a competitive sport, though. There are 1,535 comedy acts listed at this year's Fringe, and while the social media stars held their own, they'll have to do more than that to stand out on the circuit. At least they have their millions of followers to fall back on if they fail. Knock Knock is at the Pleasance Courtyard Cabaret Bar in Edinburgh until Sunday. Zookeeper pun named funniest Fringe joke TikTok comedy stars try to make it on stage Cost of living will kill stand-up, comic Porter says 'Worse than death itself': Survivors describe Libya floods Nasa reveals long-awaited findings of UFO report Russian pilot tried to shoot down British air force jet The US wants to talk to North Korea but doesnt know how New satellite images reveal Libya flood destruction Colombian begged for help but died in UK detention centre My family paid $40,000 to bring me back from the dead Why the Libyan port floods were so catastrophic Shoes to TVs - looting spree ravages war-hit Sudan Why the FBI is still searching for hundreds of Capitol rioters He ended the Bongo dynasty. Now what? Flood-hit Libyan city living through doomsday Floridas first hurricane-proof town The greatest spy novel ever written? Why is everyone crazy about Aperol © 2023 BBC. The BBC is not responsible for the content of external sites. Read about our approach to external linking"
article = text_sample
key_sentences = extract_key_sentences(article, n_sentences=10)
print(key_sentences)


They have millions of followers for their TikTok sketches.


In [74]:


def extract_top_n_sentences_with_gensim(text, n=10):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    
    # If there's less than n sentences, return them all
    if len(sentences) <= n:
        return sentences
    
    # Use Gensim's summarize function
    summary = summarize(text, ratio=n/len(sentences))
    
    return sent_tokenize(summary)

#text_sample = """[your_large_text_here]"""
top_sentences = extract_top_n_sentences_with_gensim(text_sample, 10)
for sentence in top_sentences:
    print(sentence)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mickeyshamah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'gensim.summarization'